In [36]:
!pip install langchain openai pydantic -q

In [4]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hello, I'm John."),
    AIMessage(content="Hello, John."),
    HumanMessage(content="Do you know my name."),
]

result = chat(messages)

Yes, you mentioned your name is John. How can I assist you today, John?

## PropmtTemplate

In [13]:
from langchain.prompts import PromptTemplate

template = """
以下の料理のレシピを考えてください。

料理名: {dish}
"""

prompt = PromptTemplate(
    input_variables=["dish"],
    template=template
)

result = prompt.format(dish="カレー")
print(result)


以下の料理のレシピを考えてください。

料理名: カレー



In [34]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.schema import HumanMessage, SystemMessage

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("あなたは{country}料理のプロです"),
    HumanMessagePromptTemplate.from_template("以下の大まかな料理名から、具体的な料理のレシピを考えてください。また、料理のイメージがわかる画像を検索するためのGoogle画像検索のためのURLを教えてください \n\n料理名: {dish}")
])

messages = chat_prompt.format_prompt(country="オランダ", dish="魚料理").to_messages()
messages

[SystemMessage(content='あなたはオランダ料理のプロです'),
 HumanMessage(content='以下の大まかな料理名から、具体的な料理のレシピを考えてください。また、料理のイメージがわかる画像を検索するためのGoogle画像検索のためのURLを教えてください \n\n料理名: 魚料理')]

In [35]:
chat(messages)

オランダ料理の代表的な魚料理として、「ヘリング」という料理があります。ヘリングは、生のニシンを塩漬けにして、特製のソースやタマネギと一緒に食べる料理です。

ヘリングのレシピ:
1. 新鮮なニシンをきれいに洗い、骨を取り除きます。
2. ニシンに塩をまぶし、数時間から一晩寝かせて塩漬けにします。
3. 塩漬けしたニシンを軽く水洗いし、細切りにします。
4. タマネギを薄切りにし、特製のソース（酢、砂糖、塩、こしょうを混ぜたもの）と一緒に盛り付けます。
5. レモンの輪切りやパセリを添えて、冷蔵庫で冷やしてから食べます。

Google画像検索のURL: https://www.google.com/search?q=%E3%83%98%E3%83%AA%E3%83%B3%E3%82%B0+%E3%82%AA%E3%83%A9%E3%83%B3%E3%83%80%E3%83%BB%E3%83%98%E3%83%AA%E3%83%B3%E3%82%B0&tbm=isch

AIMessage(content='オランダ料理の代表的な魚料理として、「ヘリング」という料理があります。ヘリングは、生のニシンを塩漬けにして、特製のソースやタマネギと一緒に食べる料理です。\n\nヘリングのレシピ:\n1. 新鮮なニシンをきれいに洗い、骨を取り除きます。\n2. ニシンに塩をまぶし、数時間から一晩寝かせて塩漬けにします。\n3. 塩漬けしたニシンを軽く水洗いし、細切りにします。\n4. タマネギを薄切りにし、特製のソース（酢、砂糖、塩、こしょうを混ぜたもの）と一緒に盛り付けます。\n5. レモンの輪切りやパセリを添えて、冷蔵庫で冷やしてから食べます。\n\nGoogle画像検索のURL: https://www.google.com/search?q=%E3%83%98%E3%83%AA%E3%83%B3%E3%82%B0+%E3%82%AA%E3%83%A9%E3%83%B3%E3%83%80%E3%83%BB%E3%83%98%E3%83%AA%E3%83%B3%E3%82%B0&tbm=isch', response_metadata={'finish_reason': 'stop'})

## Output parsers

In [37]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
  ingredients: list[str] = Field(description="ingredients of the dish")
  steps: list[str] = Field(description="steps to make the dish")

In [38]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)
format_instructions = parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [39]:
template = """
以下の大まかな料理名から、具体的な料理のレシピを考えてください。また、料理のイメージがわかる画像を検索するためのGoogle画像検索のためのURLを教えてください。

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={
        "format_instructions": format_instructions
    }
)

In [42]:
formatted_prompt = prompt.format(dish="カレー")

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
)

messages = [
    HumanMessage(content=formatted_prompt),
]

result = chat(messages)
print(result.content)

{
    "ingredients": [
        "玉ねぎ",
        "人参",
        "ジャガイモ",
        "牛肉",
        "カレールー",
        "水",
        "油"
    ],
    "steps": [
        "1. 鍋に油を熱し、玉ねぎを炒める。",
        "2. 人参、ジャガイモ、牛肉を加えて炒める。",
        "3. 水を加えて煮込み、カレールーを溶かす。",
        "4. 具材が柔らかくなったら完成。"
    ]
}
